# 12. Real-World Problems

In this notebook, we apply VAMOS to representative real-world scenarios:
1. **Feature Selection**: Binary optimization for datasets.
2. **Hyperparameter Tuning**: Mixed-integer optimization for ML models.
3. **Engineering Design**: Constrained optimization (Welded Beam).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import optimize
from vamos.algorithms import NSGAIIConfig
from vamos.foundation.problem.real_world.feature_selection import FeatureSelectionProblem
from vamos.foundation.problem.real_world.hyperparam import HyperparameterTuningProblem
from vamos.foundation.problem.real_world.engineering import WeldedBeamDesignProblem

plt.style.use("ggplot")
print("Libraries and problems loaded!")

## 1. Feature Selection (Binary)

We want to select a subset of features to minimize:
1. Validated Classification Error (maximize accuracy)
2. Number of Features (minimize complexity)

This uses the **Binary Knapsack**-like encoding but with an expensive evaluation function (training a model).

In [ ]:
# Create Feature Selection Problem
# Note: This requires scikit-learn. If not installed, it uses a dummy dataset.
fs_problem = FeatureSelectionProblem(dataset="breast_cancer", test_size=0.3)
print(f"Feature Selection Problem:")
print(f"  {fs_problem.n_var} Potential Features (binary)")
print(f"  {fs_problem.n_obj} Objectives (Error, Count)")

# Configure Algorithm for Binary Variables
# We strictly need BitFlip mutation and Uniform Crossover
fs_config = NSGAIIConfig.builder().pop_size(40).crossover("uniform", prob=0.9).mutation("bitflip", prob=0.05).build()

print("Starting Feature Selection Optimization...")
fs_result = optimize(
    dict(
        problem=fs_problem,
        algorithm="nsgaii",
        algorithm_config=fs_config,
        termination=("n_eval", 2000),  # Lower budget as eval is slow
        seed=42,
    )
)

print(f"Found {len(fs_result)} feature subsets")

In [ ]:
# Plot Trade-off
fs_result.plot(title="Feature Selection: Error vs Complexity")

# Inspect Best Solutions
F = fs_result.F
best_acc_idx = np.argmin(F[:, 0])  # Minimize error
fewest_feat_idx = np.argmin(F[:, 1])  # Minimize count

print("Best Accuracy Subset:")
print(f"  Error: {F[best_acc_idx, 0]:.4f}, Features: {int(F[best_acc_idx, 1])}")
print("Fewest Features Subset:")
print(f"  Error: {F[fewest_feat_idx, 0]:.4f}, Features: {int(F[fewest_feat_idx, 1])}")

## 2. Hyperparameter Tuning (Mixed)

Optimizing an SVM Classifier:
- **Real**: C (box constraint), Gamma (kernel width)
- **Integer**: Degree (polynomial kernel only)
- **Categorical**: Kernel Type (linear, rbf, poly)

This demonstrates VAMOS's ability to handle **Mixed Encodings** seamlessly.

In [ ]:
hp_problem = HyperparameterTuningProblem(dataset="breast_cancer")
print("Hyperparameter Problem:")
print(f"  Encoding: {hp_problem.encoding}")
print(f"  Real variables: C, Gamma")
print(f"  Int/Cat variables: Degree, Kernel")

# Configure for Mixed Encoding
# Note: VAMOS standard operators (SBX/PM) often auto-adapt,
# but explicitly using "mixed" if available or ensuring defaults work is key.
# For now, we reuse the generic integer-compatible config which robustly handles mixed types.
hp_config = (
    NSGAIIConfig.builder()
    .pop_size(50)
    .crossover("sbx", prob=0.9, eta=20.0)  # VAMOS SBX handles ints/real
    .mutation("pm", prob=0.2, eta=20.0)  # High mutation for exploration
    
    .build()
)

print("Tuning Hyperparameters...")
hp_result = optimize(
    dict(
        problem=hp_problem,
        algorithm="nsgaii",
        algorithm_config=hp_config,
        termination=("n_eval", 3000),
        seed=42,
    )
)

hp_result.plot(title="Hyperparameter Tuning Pareto Front")

## 3. Welded Beam Design (Engineering)

A classic constrained structural optimization problem.
- Minimize: Cost, Deflection
- Subject to: Stress, Buckling, Geometric Constraints

In [ ]:
beam_problem = WeldedBeamDesignProblem()
print("Welded Beam Problem:")
print(f"  Objectives: {beam_problem.n_obj}, Constraints: {beam_problem.n_constr}")

# Simplified Run (using defaults for Real variables)
# Since this is a standard real-valued problem (mostly), we can use the simplified API.
print("Optimizing Beam Design...")
beam_result = optimize(beam_problem, algorithm="nsgaii", pop_size=100, budget=10000, seed=42)

print(f"Found {len(beam_result)} solutions")
beam_result.plot(title="Welded Beam Design")